In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
#批次大小
batch_size=100
#训练轮输
n_batch=mnist.train.num_examples //batch_size
#神经元输出概率——过拟合处理使用
keep_probability=0.9

def weights_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#卷积操作
def conv2d(x,W):
    """
    Given an input tensor of shape `[batch, in_height, in_width, in_channels]`
    and a filter / kernel tensor of shape
    `[filter_height, filter_width, in_channels, out_channels]`
    
    Must have `strides[0] = strides[3] = 1`.  For the most common case of the same
    horizontal and vertices strides, `strides = [1, stride, stride, 1]`.
    """
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')


#池化操作
def max_pool_2x2(x):
    """
    卷积核大小：ksize=[1,x,y,1]
    """
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')



In [4]:
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])

#Reshape x to x_image:[batch, in_height, in_width, in_channels]
#-1==None,表示数量不确定
x_image=tf.reshape(x,[-1,28,28,1])



#############卷积层一###################

#filter初始化[filter_height, filter_width, in_channels, out_channels]
#32个卷积核从1个平面抽取特征
W_conv1=weights_variable([5,5,1,32])

#每个卷积核对应一个偏置值，32个卷积核
b_conv1=bias_variable([32])

h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
#池化
h_pool1=max_pool_2x2(h_conv1)


#############卷积层二###################

#5x5卷积核大小，使用64个卷积核从32个平面中提取特征
W_conv2=weights_variable([5,5,32,64])
b_conv2=bias_variable([64])

h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2=max_pool_2x2(h_conv2)



### 说明

一张28x28的图片（通道为1）在第一次卷积后还是28x28（使用的是32个卷积核，所以现在的通道为32）；
池化之后便是一张14x14(通道32)的图片

再一次卷积：14x14（通道64）；
再池化：7x7（通道64）

In [5]:

############全连接层一##############
W_fc1=weights_variable([7*7*64,1000])
b_fc1=bias_variable([1000])

#将第二次卷积层的池化结果扁平化（flat）成一维
h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
#全连接输出
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

#神经元输出概率
keep_prob=tf.placeholder(tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

############全连接层二##############

W_fc2=weights_variable([1000,10])
b_fc2=bias_variable([10])

#计算最终的输出结果
prediction=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)


In [6]:
#优化
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
train_step=tf.train.AdamOptimizer(1e-4).minimize(loss)

correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [9]:
#训练

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(11):
        for batch in range(n_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:keep_probability})
        
        acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1})
        print("Iter epoch:%s,Testing accuracy:%s" % (epoch,acc))
    


Iter epoch:0,Testing accuracy:0.9581
Iter epoch:1,Testing accuracy:0.9764
Iter epoch:2,Testing accuracy:0.9775
Iter epoch:3,Testing accuracy:0.9787
Iter epoch:4,Testing accuracy:0.9841
Iter epoch:5,Testing accuracy:0.9867
Iter epoch:6,Testing accuracy:0.9868
Iter epoch:7,Testing accuracy:0.9884
Iter epoch:8,Testing accuracy:0.9885
Iter epoch:9,Testing accuracy:0.989
Iter epoch:10,Testing accuracy:0.9884
